In [49]:
from IPython.display import Markdown
import os
from dotenv import load_dotenv
from pydantic_ai import Agent, RunContext
from dataclasses import dataclass
import httpx
from rich import print

In [ ]:
load_dotenv()

GROQ_API_KEY = os.getenv("GROQ_API_KEY")

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")  # I don't have credit, it's only to show how fallback model wotks.

# Core Concepts

This notebook explores core concepts of Pydantic AI: Agents, Dependencies, and Tools.

## Agent

An Agent is the core abstraction in Pydantic AI. It coordinates model calls, manages context, and orchestrates tool execution.

### Overview

In [ ]:
from pydantic_ai import Agent, ModelSettings
from pydantic_ai.models.groq import GroqModel
from pydantic_ai.models.fallback import FallbackModel
from pydantic_ai.models.openai import OpenAIChatModel

# Configure each model with provider-specific settings
openai_model = OpenAIChatModel(
    'gpt-5',
    settings=ModelSettings(temperature=0.2, max_tokens=100)
)
groq_model = GroqModel(
    'llama-3.3-70b-versatile',
    settings=ModelSettings(temperature=1.2, max_tokens=100)
)

# Fallback model: tries OpenAI first, then Groq if it fails
fallback_model = FallbackModel(openai_model, groq_model)
agent = Agent(fallback_model)

### Running the Agent

In [ ]:
# Execute agent with a prompt

result = await agent.run('Escreva uma história sobre exploração espacial. Português PT-BR.')

### Inspecting Results

In [ ]:
# Display the agent's final output

Markdown(result.output)

In [ ]:
# Inspect all messages: requests, responses, and model reasoning

result.all_messages()

In [ ]:
# Access the model's reasoning process

Markdown(result.all_messages()[-1].thinking)

In [ ]:
# Extract which model was used from the fallback chain

provider_name = result.all_messages()[-1].provider_name
model_name = result.all_messages()[-1].model_name

print(f"Provider: {provider_name}")
print(f"Model: {model_name}")

In [ ]:
# Extract token usage metrics

request_usage = result.all_messages()[-1].usage

print(f"Input tokens: {request_usage.input_tokens}")
print(f"Output tokens: {request_usage.output_tokens}")

### Key Concepts

- **FallbackModel**: Chains multiple models—if the first fails, it automatically tries the next
- **ModelSettings**: Configure temperature, max_tokens, and other inference parameters per model
- **RunContext**: Provides access to dependencies, messages, and request metadata during execution

## Dependencies

Dependencies enable secure injection of shared resources (HTTP clients, databases, API keys) into agent tools without exposing them to the model.

### Overview

In [ ]:
from dataclasses import dataclass
from pydantic_ai import RunContext

# Define a dependency container
@dataclass
class MyDeps:
    """Shared resources injected into agent tools"""
    api_key: str
    http_client: httpx.AsyncClient

### Configure Agent with Dependencies

In [ ]:
# Create agent with dependency type hint

agent_with_deps = Agent(
    model='groq:openai/gpt-oss-20b',
    deps_type=MyDeps,
    system_prompt='You are a helpful assistant that can fetch user information.'
)

### Define Tool Using Dependencies

In [ ]:
@agent_with_deps.tool
async def get_user_info(ctx: RunContext[MyDeps], user_id: int) -> str:
    """Fetch user information using injected HTTP client and API key"""
    # Access dependencies through context—no need to pass them explicitly
    client = ctx.deps.http_client
    api_key = ctx.deps.api_key
    
    # In production, this would make a real API call
    user_data = {
        'id': user_id,
        'name': f'User {user_id}',
        'email': f'user{user_id}@example.com',
        'authenticated': True
    }
    
    return str(user_data)

### Running Agent with Dependencies

In [ ]:
# Instantiate dependencies and pass to agent

async with httpx.AsyncClient() as client:
    deps = MyDeps(
        api_key='sk-example-key',
        http_client=client
    )
    
    result = await agent_with_deps.run(
        'Tell me about user 42.',
        deps=deps
    )

### Inspecting Results

In [ ]:
# Display agent output

Markdown(result.output)

In [ ]:
# View the entire message flow including tool calls

result.all_messages()

In [ ]:
# Extract token usage

request_usage = result.all_messages()[-1].usage

print(f"Input tokens: {request_usage.input_tokens}")
print(f"Output tokens: {request_usage.output_tokens}")

### Key Concepts

- **Dependency Injection**: Resources are passed to the agent, not hardcoded in tools
- **Backend Safety**: Sensitive data (API keys, DB connections) never reach the LLM—only tool logic sees them
- **Resource Reuse**: A single HTTP client or database connection is shared across all tool calls
- **Type Safety**: `deps_type` provides IDE autocompletion and type hints for dependency access

## Function Tools

Tools enable agents to perform actions and access external systems. Pydantic AI supports two types: `@agent.tool` (with RunContext) and `@agent.tool_plain` (without context).

### Overview

This example implements 'Jogo do Bicho' (The Animal Game), a traditional Brazilian lottery game where:
- The agent draws a random animal (1-25)
- Accesses the player's name from dependencies
- Determines if the guess matches the drawn animal

### Setup

In [151]:
import random

# Agent for 'Jogo do Bicho' game

animal_game = Agent(
    model='groq:openai/gpt-oss-20b',
    deps_type=str,  # Dependencies is just the player's name
    system_prompt=(
        'You are hosting Jogo do Bicho (The Animal Game). Draw a random animal from 1-25, '
        'check if the player\'s guess matches. If correct, celebrate their win. '
        'If wrong, tell them which animal was drawn. Always use the player\'s name in your response.'
    )
)

### Tool without Context

`@agent.tool_plain` is a simple function that doesn't need RunContext or dependencies.

In [152]:
@animal_game.tool_plain
def draw_animal() -> str:
    """Draw a random animal from the Jogo do Bicho game (1-25)."""
    animals = [
        'Avestruz', 'Águia', 'Burro', 'Borboleta', 'Cachorro',
        'Cabra', 'Carneiro', 'Camelo', 'Cobra', 'Coelho',
        'Corvos', 'Cabalo', 'Elefante', 'Estribo', 'Escorpião',
        'Espelho', 'Espingarda', 'Estátua', 'Estrela', 'Estrondo',
        'Foca', 'Formiga', 'Fruta', 'Faisão', 'Fazenda'
    ]
    drawn_number = random.randint(1, 25)
    drawn_animal = animals[drawn_number - 1]
    return f"{drawn_number} - {drawn_animal}"

### Tool with Context

`@agent.tool` has access to RunContext, allowing it to use dependencies.

In [153]:
@animal_game.tool
def get_player_name(ctx: RunContext[str]) -> str:
    """Get the player's name from dependencies."""
    # ctx.deps contains the player name (a simple string)
    return ctx.deps

### Running the Game

In [154]:
# Play the game with player's name as dependency

result = await animal_game.run(
    'My guess is Cobra (Snake)',
    deps='João'  # Player's name
)

### Inspecting Results

In [155]:
# Display game result

Markdown(result.output)

Sorry João, the drawn animal was Formiga.

In [156]:
# View all tool calls and responses

result.all_messages()

[ModelRequest(parts=[SystemPromptPart(content="You are hosting Jogo do Bicho (The Animal Game). Draw a random animal from 1-25, check if the player's guess matches. If correct, celebrate their win. If wrong, tell them which animal was drawn. Always use the player's name in your response.", timestamp=datetime.datetime(2025, 11, 4, 3, 30, 45, 299339, tzinfo=datetime.timezone.utc)), UserPromptPart(content='My guess is Cobra (Snake)', timestamp=datetime.datetime(2025, 11, 4, 3, 30, 45, 299379, tzinfo=datetime.timezone.utc))]),
 ModelResponse(parts=[ThinkingPart(content='We need to get player name, draw random animal, compare guess to drawn animal. If match, celebrate. If not, tell animal. The guess: "Cobra (Snake)". Need mapping of animal names to numbers? Not provided. We could just compare guess string to drawn animal string. But need to get drawn animal name. The draw_animal function probably returns an object with name? Not specified. Let\'s assume it returns something like {animal: "C

In [157]:
# Extract token usage

request_usage = result.all_messages()[-1].usage

print(f"Input tokens: {request_usage.input_tokens}")
print(f"Output tokens: {request_usage.output_tokens}")

Input tokens: 263

Output tokens: 111

### Key Concepts

- **`@agent.tool_plain`**: Simple function without context—ideal for stateless operations
- **`@agent.tool`**: Receives `RunContext[DepsType]`—access dependencies, model state, and request info
- **Tool Discovery**: Pydantic AI automatically discovers all decorated tools and exposes them to the model
- **Tool Results**: Tools can return strings, Pydantic models, or JSON-serializable objects
- **Async Support**: Both decorators support async functions for I/O-heavy operations

```mermaid
sequenceDiagram
    participant Agent
    participant LLM

    Agent->>LLM: Envia prompts (System + User: "My guess is 4")
    LLM-->>Agent: Decide usar a ferramenta: Call tool roll_dice()
    Agent->>Agent: Executa roll_dice() (rola o dado)
    Agent->>LLM: Retorno da ferramenta: ToolReturn "4"
    LLM-->>Agent: Decide usar outra ferramenta: Call tool get_player_name()
    Agent->>Agent: Executa get_player_name() (obtém o nome)
    Agent->>LLM: Retorno da ferramenta: ToolReturn "Anne"
    LLM-->>Agent: Constrói resposta final: ModelResponse "Congratulations Anne, ..."
```